이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Spark Session**을 하나 만든다

In [13]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark FS Partition Demo") \
        .master("local[3]") \
        .enableHiveSupport() \
        .getOrCreate()

In [14]:
!wget https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv

--2023-07-06 17:13:26--  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv
Resolving pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)... 52.218.217.81, 52.92.212.42, 3.5.81.204, ...
Connecting to pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)|52.218.217.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/csv]
Saving to: ‘appl_stock.csv.2’

appl_stock.csv.2    100%[===================>] 139.78K   440KB/s    in 0.3s    

2023-07-06 17:13:27 (440 KB/s) - ‘appl_stock.csv.2’ saved [143130/143130]



파일을 하나 다운받아서 spark managed table로 저장을 해보자

In [15]:
df = spark.read.csv("appl_stock.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [16]:
df.show(10)

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [17]:
spark.catalog.listTables()

[Table(name='appl_stock', database='default', description=None, tableType='MANAGED', isTemporary=False)]

In [18]:
# year, month 칼럼 생성

df = df.withColumn("year", year(df.Date)) \
    .withColumn("month", month(df.Date))

In [19]:
# 아래 문에서 appl_stock를 저장하기 때문에 삭제해준다
spark.sql("DROP TABLE IF EXISTS appl_stock")

DataFrame[]

In [20]:
# year, month 기준으로 파티셔닝을 해서 저장
df.write.partitionBy("year", "month").saveAsTable("appl_stock")

In [21]:
!ls -tl 

total 912
drwxr-xr-x 3 jovyan users     96 Jul  6 17:13 spark-warehouse
-rw-r--r-- 1 jovyan users  18328 Jul  6 17:13 PySpark_FileSystem_Partitioning.ipynb
drwxr-xr-x 9 jovyan users    288 Jul  6 17:07 metastore_db
-rw-r--r-- 1 jovyan users    734 Jul  6 17:07 derby.log
drwxr-xr-x 7 jovyan users    224 Jul  6 17:05 bucket_partitioning
drwxr-xr-x 8 jovyan users    256 Jul  6 15:22 join_broadcast_join
drwxr-xr-x 5 jovyan users    160 Jul  6 15:09 wordcount
-rw-r--r-- 1 jovyan users  20898 Jul  6 15:09 PySpark_Schema_Evolution.ipynb
-rw-r--r-- 1 jovyan users  11503 Jul  6 08:19 PySpark_Write_실습.ipynb
drwxr-xr-x 5 jovyan users    160 Jul  6 08:17 dataOutput
-rw-r--r-- 1 jovyan users   1640 Jul  4 12:19 bucketDemo.py
-rw-r--r-- 1 jovyan users  30374 Jul  4 12:19 PySpark_ML_실습2.ipynb
-rw-r--r-- 1 jovyan users  32836 Jul  4 12:19 PySpark_ML_실습1.ipynb
-rw-r--r-- 1 jovyan users  71580 Jan 25 22:11 schema3.parquet
-rw-r--r-- 1 jovyan users  71580 Jan 25 22:11 schema3.parquet.1
-rw-r--r-- 1 jovya

In [9]:
!ls -tl spark-warehouse/

total 0
drwxr-xr-x 11 jovyan users 352 Jul  6 17:07 appl_stock


In [10]:
!ls -tl spark-warehouse/appl_stock/

total 0
-rw-r--r--  1 jovyan users   0 Jul  6 17:07  _SUCCESS
drwxr-xr-x 14 jovyan users 448 Jul  6 17:07 'year=2016'
drwxr-xr-x 14 jovyan users 448 Jul  6 17:07 'year=2015'
drwxr-xr-x 14 jovyan users 448 Jul  6 17:07 'year=2014'
drwxr-xr-x 14 jovyan users 448 Jul  6 17:07 'year=2013'
drwxr-xr-x 14 jovyan users 448 Jul  6 17:07 'year=2012'
drwxr-xr-x 14 jovyan users 448 Jul  6 17:07 'year=2011'
drwxr-xr-x 14 jovyan users 448 Jul  6 17:07 'year=2010'


In [11]:
!ls -tl spark-warehouse/appl_stock/year\=2010/

total 0
drwxr-xr-x 4 jovyan users 128 Jul  6 17:07 'month=12'
drwxr-xr-x 4 jovyan users 128 Jul  6 17:07 'month=11'
drwxr-xr-x 4 jovyan users 128 Jul  6 17:07 'month=10'
drwxr-xr-x 4 jovyan users 128 Jul  6 17:07 'month=9'
drwxr-xr-x 4 jovyan users 128 Jul  6 17:07 'month=8'
drwxr-xr-x 4 jovyan users 128 Jul  6 17:07 'month=7'
drwxr-xr-x 4 jovyan users 128 Jul  6 17:07 'month=6'
drwxr-xr-x 4 jovyan users 128 Jul  6 17:07 'month=5'
drwxr-xr-x 4 jovyan users 128 Jul  6 17:07 'month=4'
drwxr-xr-x 4 jovyan users 128 Jul  6 17:07 'month=3'
drwxr-xr-x 4 jovyan users 128 Jul  6 17:07 'month=2'
drwxr-xr-x 4 jovyan users 128 Jul  6 17:07 'month=1'


In [12]:
!ls -tl spark-warehouse/appl_stock/year\=2010/month\=12/

total 4
-rw-r--r-- 1 jovyan users 3027 Jul  6 17:07 part-00000-16f22434-ccb8-4395-b4c4-770bce4a5007.c000.snappy.parquet


### How to Read from Partitioned table

In [15]:
df = spark.read.table("appl_stock").where("year = 2016 and month = 12")

In [16]:
df.show(10)

+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|               Date|              Open|              High|       Low|     Close|  Volume|         Adj Close|year|month|
+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|2016-12-01 00:00:00|        110.370003|        110.940002|109.029999|109.489998|37086900|        109.017344|2016|   12|
|2016-12-02 00:00:00|109.16999799999999|        110.089996|108.849998|109.900002|26528000|        109.425578|2016|   12|
|2016-12-05 00:00:00|             110.0|        110.029999|    108.25|109.110001|34324500|108.63898700000001|2016|   12|
|2016-12-06 00:00:00|             109.5|        110.360001|109.190002|109.949997|26195500|109.47535800000001|2016|   12|
|2016-12-07 00:00:00|        109.260002|        111.190002|109.160004|111.029999|29998700|        110.550697|2016|   12|
|2016-12-08 00:00:00|        110

In [17]:
spark.sql("SELECT * FROM appl_stock WHERE year = 2016 and month = 12").show(10)

+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|               Date|              Open|              High|       Low|     Close|  Volume|         Adj Close|year|month|
+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|2016-12-01 00:00:00|        110.370003|        110.940002|109.029999|109.489998|37086900|        109.017344|2016|   12|
|2016-12-02 00:00:00|109.16999799999999|        110.089996|108.849998|109.900002|26528000|        109.425578|2016|   12|
|2016-12-05 00:00:00|             110.0|        110.029999|    108.25|109.110001|34324500|108.63898700000001|2016|   12|
|2016-12-06 00:00:00|             109.5|        110.360001|109.190002|109.949997|26195500|109.47535800000001|2016|   12|
|2016-12-07 00:00:00|        109.260002|        111.190002|109.160004|111.029999|29998700|        110.550697|2016|   12|
|2016-12-08 00:00:00|        110